<a href="https://colab.research.google.com/github/francoriboratig/TP2-Datos/blob/master/Perceptron_Multicapa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as skl
from sklearn import model_selection
import keras as krs
import tensorflow as tsf
from sklearn.preprocessing import StandardScaler
from google.colab import files

url1 = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/train-features.csv'
url2 = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/test-features.csv'

train = pd.read_csv(url1)
test = pd.read_csv(url2)

train.head(5)

,Unnamed: 0,id,keyword,location,text,target,character_count,word_count,contains_word_fire,contains_word_storm,contains_word_flood,contains_word_death,contains_word_love,has_mentions,amount_of_exclamation_marks,amount_of_interrogation_marks,has_link,stopwords_count,contains_countries,contains_cities
0,0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,13,0,0,0,0,0,False,0,0,False,7,False,False
1,1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,7,1,0,0,0,0,False,0,0,False,3,True,False
2,2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,22,0,0,0,0,0,False,0,0,False,13,False,False
3,3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,8,1,0,0,0,0,False,0,0,False,2,False,False
4,4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,16,1,0,0,0,0,False,0,0,False,10,False,False


In [45]:
test.head(5)

,Unnamed: 0,id,keyword,location,text,character_count,word_count,contains_word_fire,contains_word_storm,contains_word_flood,contains_word_death,contains_word_love,has_mentions,amount_of_exclamation_marks,amount_of_interrogation_marks,has_link,stopwords_count,contains_countries,contains_cities
0,0,0,NaN,NaN,Just happened a terrible car crash,34,6,0,0,0,0,0,False,0,0,False,4,False,False
1,1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",64,9,0,0,0,0,0,False,0,0,False,7,False,False
2,2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",96,19,1,0,0,0,0,False,0,0,False,10,False,False
3,3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,40,4,1,0,0,0,0,False,0,0,False,3,False,True
4,4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,45,8,0,0,0,0,0,False,0,0,False,6,True,False


In [46]:
test['character_count'] = test['text'].apply(len)

def word_count(text):
  return len(text.split())
  
test['word_count'] = test['text'].apply(word_count)
test.head(5)

,Unnamed: 0,id,keyword,location,text,character_count,word_count,contains_word_fire,contains_word_storm,contains_word_flood,contains_word_death,contains_word_love,has_mentions,amount_of_exclamation_marks,amount_of_interrogation_marks,has_link,stopwords_count,contains_countries,contains_cities
0,0,0,NaN,NaN,Just happened a terrible car crash,34,6,0,0,0,0,0,False,0,0,False,4,False,False
1,1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",64,9,0,0,0,0,0,False,0,0,False,7,False,False
2,2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",96,19,1,0,0,0,0,False,0,0,False,10,False,False
3,3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,40,4,1,0,0,0,0,False,0,0,False,3,False,True
4,4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,45,8,0,0,0,0,0,False,0,0,False,6,True,False


In [47]:
#data_target = krs.utils.np_utils.to_categorical(train['target'].values)
data_target = train[['target']]
data_train=train.drop(['Unnamed: 0', 'id', 'keyword', 'location', 'text', 'target'],axis = 1)
data_test=test.drop(['Unnamed: 0', 'id', 'keyword', 'location', 'text'],axis = 1)

scaler = StandardScaler()
scaler.fit(data_train)

data_train=scaler.transform(data_train)
data_test=scaler.transform(data_test)

x_train, x_validation, y_train, y_validation = model_selection.train_test_split(data_train, data_target, test_size=0.3, random_state=0)

model=krs.Sequential()
model.add(krs.layers.normalization.BatchNormalization(input_shape=tuple([x_train.shape[1]])))
model.add(krs.layers.Dense(32, input_dim=14, activation='relu'))
model.add(krs.layers.core.Dropout(rate=0.5))
model.add(krs.layers.normalization.BatchNormalization())
model.add(krs.layers.Dense(32, activation='relu'))
model.add(krs.layers.core.Dropout(rate=0.5))
model.add(krs.layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=900, batch_size=512, validation_data=(x_validation, y_validation))






Train on 5329 samples, validate on 2284 samples
Epoch 1/900
5329/5329 [==============================] - 0s 68us/step - loss: 0.8277 - accuracy: 0.5281 - val_loss: 0.6618 - val_accuracy: 0.5898
Epoch 2/900
5329/5329 [==============================] - 0s 8us/step - loss: 0.7710 - accuracy: 0.5667 - val_loss: 0.6425 - val_accuracy: 0.6313
Epoch 3/900
5329/5329 [==============================] - 0s 7us/step - loss: 0.7493 - accuracy: 0.5708 - val_loss: 0.6315 - val_accuracy: 0.6462
Epoch 4/900
5329/5329 [==============================] - 0s 7us/step - loss: 0.7206 - accuracy: 0.5868 - val_loss: 0.6237 - val_accuracy: 0.6594
Epoch 5/900
5329/5329 [==============================] - 0s 7us/step - loss: 0.7158 - accuracy: 0.5967 - val_loss: 0.6196 - val_accuracy: 0.6699
Epoch 6/900
5329/5329 [==============================] - 0s 8us/step - loss: 0.6991 - accuracy: 0.5952 - val_loss: 0.6163 - val_accuracy: 0.6751
Epoch 7/900
5329/5329 [==============================] - 0s 7us/step - loss: 0.69

In [48]:
loss,precision = model.evaluate(x_train, y_train)
print('Precisión: %.2f' % (precision*100))

5329/5329 [==============================] - 0s 21us/step
Precisión: 71.50


In [49]:
loss

0.5591102682597912

In [50]:
prediction=model.predict_classes(data_test, batch_size=512)
prediction

array([[0],
       [0],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=int32)

In [51]:
test['target'] = prediction

test.head(5)

,Unnamed: 0,id,keyword,location,text,character_count,word_count,contains_word_fire,contains_word_storm,contains_word_flood,contains_word_death,contains_word_love,has_mentions,amount_of_exclamation_marks,amount_of_interrogation_marks,has_link,stopwords_count,contains_countries,contains_cities,target
0,0,0,NaN,NaN,Just happened a terrible car crash,34,6,0,0,0,0,0,False,0,0,False,4,False,False,0
1,1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",64,9,0,0,0,0,0,False,0,0,False,7,False,False,0
2,2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",96,19,1,0,0,0,0,False,0,0,False,10,False,False,1
3,3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,40,4,1,0,0,0,0,False,0,0,False,3,False,True,1
4,4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,45,8,0,0,0,0,0,False,0,0,False,6,True,False,0


In [53]:
export = pd.DataFrame()
export['id']=test['id']
export['target'] = test['target']
export= export.set_index('id')
export.to_csv('submit.csv')
#files.download('submit.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>